# 3. Building Compositional Space Complexes to Express Elaborate Design Problems

> ***(before running this code, please consult Quick Start to make sure everything is set up)***

> You will need the same dependencies as in [Tutorial No.2](02.AdditiveManufacturingPathPlanning.ipynb) (i.e., `pycalphad` and `pathfinding`), and additionally `igraph` in order to efficiently visualize the examples presented in this tutorial. **If you are running this in `nimplex`'s Codespaces, everything has been pre-installed for you.**

**In this tutorial, we will build upon `nimplex`'s graph representations and leverage our ability to combine ("stitch") them together without affecting the way we express the problem at hand and in an effortless fashion to dramatically speed up materials exploration.** In particular, we will show how to:

1. **Plot graphs** (as in mathematical objects, not any figures) using [`igraph`](https://igraph.org/), which can be done quite directly using `nimplex`'s point grid and neighbor lists `edges = [(i,n) for i in range(len(gridAtt)) for n in nList[i]]`

2. **Identify ordered subsystems in `nimplex` grids, like `A-B-C` and `C-A-B` within `A-B-C-D-E` and `D-C-G-F-A-H-B` to establish connectivity between them.** We will use this to combine 3 4-component systems (tetrahedra) by 2 3-component subsystems (triangles) to create a chain.

3. Identify all subspaces of a given order in low dimensional and high dimensional spaces (e.g., all quantized compositions of any 3 things out of N) and combine ("stitch") them together to form a **simplex graphs that intersect themselves in 3D because of high dimensionality, yet still have the same (graph) structure**.

4. **Construct a graph complex to explore all ternary combinations of `["Ti50Zr50", "Hf95Ti5", "NbTaWHf", "Mo80Nb10W10", "TiTa2", "Nb96Mo3W1", "Zr49 Hf1 Mo50"]` (7 alloys)** under equilibrium phase constraint (relatively expensive to compute) and then explore the space with an additional low-cost screenig constraint (RMSAD - alloy strenght surrogate) that could also be an ML surrogate.

In [1]:
import nimplex
from utils import stitching
from IPython.display import clear_output
from itertools import combinations

In [2]:
import igraph as ig
import plotly.graph_objs as go
import numpy as np
import random
random.seed(123)

## Plotting 4-Component Simplex Graph with `igraph`

In [3]:
dim = 4
ndiv = 6
gridAtt, nList = nimplex.simplex_graph_py(dim, ndiv)

In [4]:
edges = []
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i,n))

In [5]:
def plotGraph(edges: list):
    # Lets generate some seed positions for the nodes
    G = ig.Graph(edges)
    layout = G.layout_kamada_kawai(dim=3)
    layout_array = np.array(layout.coords)
    # Create edge traces
    edge_traces = []
    for edge in G.es:
        start, end = edge.tuple
        x0, y0, z0 = layout_array[start]
        x1, y1, z1 = layout_array[end]
        width = 5
        edge_trace = go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1],
                                line=dict(width=width, color='lightgray'),
                                opacity=0.2,
                                hoverinfo='none', mode='lines')
        edge_traces.append(edge_trace)

    node_trace = go.Scatter3d(x=layout_array[:, 0], y=layout_array[:, 1], z=layout_array[:, 2],
                            mode='markers',
                            marker=dict(size=3, 
                                        opacity=0.5,
                                        line=None,
                                        color='blue'),
                            
                            text=[f"Node {i}" for i in range(len(gridAtt))],
                            hoverinfo='text')

    # Create figure
    fig = go.Figure(data=edge_traces + [node_trace])

    # Update layout
    fig.update_layout(
        title='3D Graph with Feasibility Types',
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        showlegend=False,
        margin=dict(l=0, r=0, b=0, t=0),
        width=800
    )

    fig.show()

In [6]:
#plotGraph(edges)
clear_output()

## Combining ("Stitching") 3 4-Component Systems into A Complex

In [7]:
sys1 = ["A", "B", "C", "D"]
sys2 = ["B", "C", "D", "E"]
sys3 = ["B", "D", "E", "F"]

In [8]:
offset = len(gridAtt)
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i+offset,n+offset))
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i+offset*2,n+offset*2))

In [9]:
#plotGraph(edges)
clear_output()

In [10]:
stitchingPoints = stitching.findStitchingPoints_py(dim, ndiv, components=["A", "B", "C", "D"])
print(stitchingPoints["A-B"])

[83, 82, 79, 73, 63, 48, 27]


In [11]:
stitchingPoints1 = stitching.findStitchingPoints_py(dim, ndiv, components=sys1, offset=0)
stitchingPoints2 = stitching.findStitchingPoints_py(dim, ndiv, components=sys2, offset=len(gridAtt))
stitchingPoints3 = stitching.findStitchingPoints_py(dim, ndiv, components=sys3, offset=len(gridAtt)*2)

In [12]:
for i, j in zip(stitchingPoints1["B-C-D"], stitchingPoints2["B-C-D"]):
    edges.append((i, j))

In [13]:
for i, j in zip(stitchingPoints2["B-D-E"], stitchingPoints3["B-D-E"]):
    edges.append((i, j))

In [14]:
#plotGraph(edges)
clear_output()

## Combining ("Stitching") 9 3-Component Systems into A Complex
### Disregard this section for now Adam

In [15]:
dim = 3
ndiv = 6
nsys = 9
gridAtt, nList = nimplex.simplex_graph_py(dim, ndiv)
edges = []
for i in range(len(gridAtt)):
    for n in nList[i]:
        edges.append((i,n))

In [16]:
len(gridAtt)

28

In [17]:
offset = len(gridAtt)
for sys in range(1,nsys+1):
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset*sys,n+offset*sys))

In [18]:
#plotGraph(edges)
clear_output()

In [19]:
import pandas as pd

In [20]:
from pycalphad import Database

In [21]:
dbCrNiV = Database("ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB")
dbCrFeTi = Database("ammap/databases/Cr-Fe-Ti_wang2017.tdb")
dbCrFeNi = Database("ammap/databases/Cr-Fe-Ni_miettinen1999.tdb")
dbCrNiTi= Database("ammap/databases/Cr-Ni-Ti_huang2018.tdb")
dbCrTiV= Database("ammap/databases/Cr-Ti-V_ghosh2002.tdb")
dbFeNiTi= Database("ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb")
dbFeNiV= Database("ammap/databases/Fe-Ni-V_zhao2014.tdb")
dbFeTiV= Database("ammap/databases/Fe-Ti-V_guo2012.TDB")
dbNiTiV= Database("ammap/databases/Ni-Ti-V_zou2018.tdb")
CrNiV = ['Cr', 'Ni', 'V']
CrFeTi = ['Cr', 'Fe', 'Ti']
CrFeNi = ['Cr', 'Fe', 'Ni']
CrNiTi = ['Cr', 'Ni', 'Ti']
CrTiV = ['Cr', 'Ti', 'V']
FeNiTi = ['Fe', 'Ni', 'Ti']
FeNiV = ['Fe', 'Ni', 'V']
FeTiV = ['Fe', 'Ti', 'V']
NiTiV = ['Ni', 'Ti', 'V']


/ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/pycalphad/io/tdb.py:293: UserWarning: The type definition character `&` in `TYPE_DEFINITION & GES A_P_D A2_BCC MAGNETIC -1.0 4.00000E-01 ` is not used by any phase.
  warnings.warn(f"The type definition character `{typechar}` in `TYPE_DEFINITION {typechar} {line}` is not used by any phase.")


In [22]:
stitchingPointsCrNiV = stitching.findStitchingPoints_py(dim, ndiv, components=CrNiV, offset=0)
stitchingPointsCrFeTi = stitching.findStitchingPoints_py(dim, ndiv, components=CrFeTi, offset=len(gridAtt))
stitchingPointsCrFeNi = stitching.findStitchingPoints_py(dim, ndiv, components=CrFeNi, offset=len(gridAtt)*2)
stitchingPointsCrNiTi = stitching.findStitchingPoints_py(dim, ndiv, components=CrNiTi, offset=len(gridAtt)*3)
stitchingPointsCrTiV = stitching.findStitchingPoints_py(dim, ndiv, components=CrTiV, offset=len(gridAtt)*4)
stitchingPointsFeNiTi = stitching.findStitchingPoints_py(dim, ndiv, components=FeNiTi, offset=len(gridAtt)*5)
stitchingPointsFeNiV = stitching.findStitchingPoints_py(dim, ndiv, components=FeNiV, offset=len(gridAtt)*6)
stitchingPointsFeTiV = stitching.findStitchingPoints_py(dim, ndiv, components=FeTiV, offset=len(gridAtt)*7)
stitchingPointsNiTiV = stitching.findStitchingPoints_py(dim, ndiv, components=NiTiV, offset=len(gridAtt)*8)

## Finding and Combining 3-Component Subspaces in 4-Component Space

In [23]:
from itertools import combinations
elements = ["A", "B", "C", "D"]
ternaries = list(combinations(elements, 3))
print(ternaries)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D')]


In [24]:
gridAtt, nList = nimplex.simplex_graph_py(3,12)

In [25]:
edges = []
# Ternaries
for i, ternary in enumerate(ternaries):
    offset = i*len(gridAtt)
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset,n+offset))

In [26]:
#plotGraph(edges)
clear_output()

In [27]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('A', 'B'): occurs between ternary [(0, 1)]
('A', 'C'): occurs between ternary [(0, 2)]
('B', 'C'): occurs between ternary [(0, 3)]
('A', 'D'): occurs between ternary [(1, 2)]
('B', 'D'): occurs between ternary [(1, 3)]
('C', 'D'): occurs between ternary [(2, 3)]


In [28]:
for stitchingBinary, ternaryPair in stitchingBinaries.items():
    ternary1, ternary2 = ternaryPair[0][0], ternaryPair[0][1]
    stitching1 = stitching.findStitchingPoints_py(
        3, 12, 
        components=ternaries[ternary1],
        offset=ternary1*len(gridAtt)
        )["-".join(stitchingBinary)]
    stitching2 = stitching.findStitchingPoints_py(
        3, 12, 
        components=ternaries[ternary2],
        offset=ternary2*len(gridAtt)
        )["-".join(stitchingBinary)]
    print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
    for i, j in zip(stitching1, stitching2):
        edges.append((i, j))

Stitching 0 and 1 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103]
Stitching 0 and 2 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 0 and 3 at ('B', 'C') from [12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0] to [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285]
Stitching 1 and 2 at ('A', 'D') from [181, 179, 176, 172, 167, 161, 154, 146, 137, 127, 116, 104, 91] to [272, 270, 267, 263, 258, 252, 245, 237, 228, 218, 207, 195, 182]
Stitching 1 and 3 at ('B', 'D') from [103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91] to [363, 361, 358, 354, 349, 343, 336, 328, 319, 309, 298, 286, 273]
Stitching 2 and 3 at ('C', 'D') from [194, 193, 192, 191, 190, 189, 188, 187, 186, 185, 184, 183, 182] to [285, 284, 283, 282, 281, 280, 279, 278, 277, 276, 275, 274, 273]


In [29]:
#plotGraph(edges)
clear_output()

## In Higher Dimensional Cases (Stretched 2D Spaces Crossing Each Other in 3D)

In [30]:
from itertools import combinations
elements = ["A", "B", "C", "D", "E"]
ternaries = list(combinations(elements, 3))
print(ternaries)

[('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'B', 'E'), ('A', 'C', 'D'), ('A', 'C', 'E'), ('A', 'D', 'E'), ('B', 'C', 'D'), ('B', 'C', 'E'), ('B', 'D', 'E'), ('C', 'D', 'E')]


In [31]:
gridAtt, nList = nimplex.simplex_graph_py(3,12)

In [32]:
edges = []
# Ternaries
for i, ternary in enumerate(ternaries):
    offset = i*len(gridAtt)
    for i in range(len(gridAtt)):
        for n in nList[i]:
            edges.append((i+offset,n+offset))


In [33]:
#plotGraph(edges)
clear_output()

In [34]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('A', 'B'): occurs between ternary [(0, 1), (0, 2), (1, 2)]
('A', 'C'): occurs between ternary [(0, 3), (0, 4), (3, 4)]
('B', 'C'): occurs between ternary [(0, 6), (0, 7), (6, 7)]
('A', 'D'): occurs between ternary [(1, 3), (1, 5), (3, 5)]
('B', 'D'): occurs between ternary [(1, 6), (1, 8), (6, 8)]
('A', 'E'): occurs between ternary [(2, 4), (2, 5), (4, 5)]
('B', 'E'): occurs between ternary [(2, 7), (2, 8), (7, 8)]
('C', 'D'): occurs between ternary [(3, 6), (3, 9), (6, 9)]
('C', 'E'): occurs between ternary [(4, 7), (4, 9), (7, 9)]
('D', 'E'): occurs between ternary [(5, 8), (5, 9), (8, 9)]


In [35]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, 12, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, 12, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))

Stitching 0 and 1 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103]
Stitching 0 and 2 at ('A', 'B') from [90, 89, 87, 84, 80, 75, 69, 62, 54, 45, 35, 24, 12] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 1 and 2 at ('A', 'B') from [181, 180, 178, 175, 171, 166, 160, 153, 145, 136, 126, 115, 103] to [272, 271, 269, 266, 262, 257, 251, 244, 236, 227, 217, 206, 194]
Stitching 0 and 3 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285]
Stitching 0 and 4 at ('A', 'C') from [90, 88, 85, 81, 76, 70, 63, 55, 46, 36, 25, 13, 0] to [454, 453, 451, 448, 444, 439, 433, 426, 418, 409, 399, 388, 376]
Stitching 3 and 4 at ('A', 'C') from [363, 362, 360, 357, 353, 348, 342, 335, 327, 318, 308, 297, 285] to [454, 453, 451, 448, 444, 439, 433, 426, 418, 409, 399, 388, 376]
Stitching 0 and 6 at (

Stitching 1 and 3 at ('A', 'D') from [181, 179, 176, 172, 167, 161, 154, 146, 137, 127, 116, 104, 91] to [363, 361, 358, 354, 349, 343, 336, 328, 319, 309, 298, 286, 273]
Stitching 1 and 5 at ('A', 'D') from [181, 179, 176, 172, 167, 161, 154, 146, 137, 127, 116, 104, 91] to [545, 544, 542, 539, 535, 530, 524, 517, 509, 500, 490, 479, 467]
Stitching 3 and 5 at ('A', 'D') from [363, 361, 358, 354, 349, 343, 336, 328, 319, 309, 298, 286, 273] to [545, 544, 542, 539, 535, 530, 524, 517, 509, 500, 490, 479, 467]
Stitching 1 and 6 at ('B', 'D') from [103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91] to [636, 634, 631, 627, 622, 616, 609, 601, 592, 582, 571, 559, 546]
Stitching 1 and 8 at ('B', 'D') from [103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91] to [818, 817, 815, 812, 808, 803, 797, 790, 782, 773, 763, 752, 740]
Stitching 6 and 8 at ('B', 'D') from [636, 634, 631, 627, 622, 616, 609, 601, 592, 582, 571, 559, 546] to [818, 817, 815, 812, 808, 803, 797, 790, 782, 773, 763

In [36]:
#plotGraph(edges)
clear_output()

## Exploring Complex Formed by All 3-Component Alloy Subsystems of 5-Component Space (w. Infeasibility Gliding) #THIS ONE ADAM

In [37]:
elementalSpaceComponents = ["Cr","Fe","Ni","Ti","V"]
#attainableSpaceComponents = ["SS304L", "Ni", "Cr", "V", "Ti"]
attainableSpaceComponents = ["Cr", "Fe", "Ni", "Ti", "V"]
attainableSpaceComponentPositions = [
    #"Cr","Fe","Ni","Ti","V"
    #[0.1993865031, 0.7044989775,0.096114519430,0,0],
    [1,0,0,0,0],
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0,0,0,1,0],
    [0,0,0,0,1]
]    
ternaries = list(combinations(attainableSpaceComponents, 3))
ternaries_CompPos = list(combinations(attainableSpaceComponentPositions, 3))
ndiv = 32
#ndiv = 5
gridAtt, nList = nimplex.simplex_graph_py(3, ndiv)

for tern, terncp in zip(ternaries, ternaries_CompPos):
    print(f"{str(tern):<40} -> {terncp}")

('Cr', 'Fe', 'Ni')                       -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0])
('Cr', 'Fe', 'Ti')                       -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0])
('Cr', 'Fe', 'V')                        -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1])
('Cr', 'Ni', 'Ti')                       -> ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Cr', 'Ni', 'V')                        -> ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Cr', 'Ti', 'V')                        -> ([1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
('Fe', 'Ni', 'Ti')                       -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Fe', 'Ni', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Fe', 'Ti', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
('Ni', 'Ti', 'V')                        -> ([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])


In [38]:
# Edges list for graph plotting and path finding purposes
edges = []
# Connectivity list within each subsystem
graphN = [[] for i in range(len(gridAtt * len(ternaries)))]
# Connectivity list between subsystems
graphNS = [[] for i in range(len(graphN))]
compositions = []
compositions_with_id = []  # List to store compositions with their identifiers
ternaries_with_id = []  # New list to store ternaries_CompPos with their identifiers

# Iterate over ternaries
for i, terncp in enumerate(ternaries_CompPos):
    ternaries_with_id.append((terncp, i))  # Add terncp and its id to the new list
    
    offset = i*len(gridAtt)
    for j in range(len(gridAtt)):
        for n in nList[j]:
            edges.append((j+offset,n+offset))
            graphN[j+offset].append(n+offset)
    gridAttTemp, gridElTemp = nimplex.embeddedpair_simplex_grid_fractional_py(terncp, ndiv)
    compositions += gridElTemp
    
    # Attach identifier to each composition
    compositions_with_id.extend([(comp, i) for comp in gridElTemp])

In [39]:
# Example: Access ternaries_CompPos with their identifiers
for terncp, ternary_id in ternaries_with_id:
    print(f"Ternary composition position {terncp} has id {ternary_id}")

Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0]) has id 0
Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0]) has id 1
Ternary composition position ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]) has id 2
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]) has id 3
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]) has id 4
Ternary composition position ([1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 5
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]) has id 6
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1]) has id 7
Ternary composition position ([0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 8
Ternary composition position ([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]) has id 9


In [40]:
mapping = {}
for ternary, id in ternaries_with_id:
    composition_key = ''.join(elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary))))
    mapping[composition_key] = id

In [41]:
mapping = {}
for ternary, id in ternaries_with_id:
    composition_key = ''.join(elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary))))
    individual_elements = [elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary)))]
    mapping[composition_key] = {
        'id': id,
        'elements': individual_elements
    }

# Print the mapping to see the result
for key, value in mapping.items():
    print(f"Combination: {key}, ID: {value['id']}, Elements: {value['elements']}")

Combination: CrFeNi, ID: 0, Elements: ['Cr', 'Fe', 'Ni']
Combination: CrFeTi, ID: 1, Elements: ['Cr', 'Fe', 'Ti']
Combination: CrFeV, ID: 2, Elements: ['Cr', 'Fe', 'V']
Combination: CrNiTi, ID: 3, Elements: ['Cr', 'Ni', 'Ti']
Combination: CrNiV, ID: 4, Elements: ['Cr', 'Ni', 'V']
Combination: CrTiV, ID: 5, Elements: ['Cr', 'Ti', 'V']
Combination: FeNiTi, ID: 6, Elements: ['Fe', 'Ni', 'Ti']
Combination: FeNiV, ID: 7, Elements: ['Fe', 'Ni', 'V']
Combination: FeTiV, ID: 8, Elements: ['Fe', 'Ti', 'V']
Combination: NiTiV, ID: 9, Elements: ['Ni', 'Ti', 'V']


In [42]:
mapping

{'CrFeNi': {'id': 0, 'elements': ['Cr', 'Fe', 'Ni']},
 'CrFeTi': {'id': 1, 'elements': ['Cr', 'Fe', 'Ti']},
 'CrFeV': {'id': 2, 'elements': ['Cr', 'Fe', 'V']},
 'CrNiTi': {'id': 3, 'elements': ['Cr', 'Ni', 'Ti']},
 'CrNiV': {'id': 4, 'elements': ['Cr', 'Ni', 'V']},
 'CrTiV': {'id': 5, 'elements': ['Cr', 'Ti', 'V']},
 'FeNiTi': {'id': 6, 'elements': ['Fe', 'Ni', 'Ti']},
 'FeNiV': {'id': 7, 'elements': ['Fe', 'Ni', 'V']},
 'FeTiV': {'id': 8, 'elements': ['Fe', 'Ti', 'V']},
 'NiTiV': {'id': 9, 'elements': ['Ni', 'Ti', 'V']}}

In [43]:
# def find_duplicates(lst):
#     duplicates_counts = {}
#     for item in lst:
#         # Use the string representation for complex/unhashable types
#         item_str = str(item)
#         if item_str in duplicates_counts:
#             duplicates_counts[item_str] += 1
#         else:
#             duplicates_counts[item_str] = 1

#     # Filter out items that only appear once
#     duplicates = {item: count for item, count in duplicates_counts.items() if count > 1}
    
#     if not duplicates:
#         return "No duplicates found in the list."
#     else:
#         result = "Duplicates found:"
#         for item, count in duplicates.items():
#             result += f"{item} appears {count} times"
#         return result.strip()

In [44]:
# find_duplicates(compositions)

In [45]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

('Cr', 'Fe'): occurs between ternary [(0, 1), (0, 2), (1, 2)]
('Cr', 'Ni'): occurs between ternary [(0, 3), (0, 4), (3, 4)]
('Fe', 'Ni'): occurs between ternary [(0, 6), (0, 7), (6, 7)]
('Cr', 'Ti'): occurs between ternary [(1, 3), (1, 5), (3, 5)]
('Fe', 'Ti'): occurs between ternary [(1, 6), (1, 8), (6, 8)]
('Cr', 'V'): occurs between ternary [(2, 4), (2, 5), (4, 5)]
('Fe', 'V'): occurs between ternary [(2, 7), (2, 8), (7, 8)]
('Ni', 'Ti'): occurs between ternary [(3, 6), (3, 9), (6, 9)]
('Ni', 'V'): occurs between ternary [(4, 7), (4, 9), (7, 9)]
('Ti', 'V'): occurs between ternary [(5, 8), (5, 9), (8, 9)]


In [46]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            edges.append((i, j))
            graphNS[i].append(j)

Stitching 0 and 1 at ('Cr', 'Fe') from [560, 559, 557, 554, 550, 545, 539, 532, 524, 515, 505, 494, 482, 469, 455, 440, 424, 407, 389, 370, 350, 329, 307, 284, 260, 235, 209, 182, 154, 125, 95, 64, 32] to [1121, 1120, 1118, 1115, 1111, 1106, 1100, 1093, 1085, 1076, 1066, 1055, 1043, 1030, 1016, 1001, 985, 968, 950, 931, 911, 890, 868, 845, 821, 796, 770, 743, 715, 686, 656, 625, 593]


Stitching 0 and 2 at ('Cr', 'Fe') from [560, 559, 557, 554, 550, 545, 539, 532, 524, 515, 505, 494, 482, 469, 455, 440, 424, 407, 389, 370, 350, 329, 307, 284, 260, 235, 209, 182, 154, 125, 95, 64, 32] to [1682, 1681, 1679, 1676, 1672, 1667, 1661, 1654, 1646, 1637, 1627, 1616, 1604, 1591, 1577, 1562, 1546, 1529, 1511, 1492, 1472, 1451, 1429, 1406, 1382, 1357, 1331, 1304, 1276, 1247, 1217, 1186, 1154]
Stitching 1 and 2 at ('Cr', 'Fe') from [1121, 1120, 1118, 1115, 1111, 1106, 1100, 1093, 1085, 1076, 1066, 1055, 1043, 1030, 1016, 1001, 985, 968, 950, 931, 911, 890, 868, 845, 821, 796, 770, 743, 715, 686, 656, 625, 593] to [1682, 1681, 1679, 1676, 1672, 1667, 1661, 1654, 1646, 1637, 1627, 1616, 1604, 1591, 1577, 1562, 1546, 1529, 1511, 1492, 1472, 1451, 1429, 1406, 1382, 1357, 1331, 1304, 1276, 1247, 1217, 1186, 1154]


Stitching 0 and 3 at ('Cr', 'Ni') from [560, 558, 555, 551, 546, 540, 533, 525, 516, 506, 495, 483, 470, 456, 441, 425, 408, 390, 371, 351, 330, 308, 285, 261, 236, 210, 183, 155, 126, 96, 65, 33, 0] to [2243, 2242, 2240, 2237, 2233, 2228, 2222, 2215, 2207, 2198, 2188, 2177, 2165, 2152, 2138, 2123, 2107, 2090, 2072, 2053, 2033, 2012, 1990, 1967, 1943, 1918, 1892, 1865, 1837, 1808, 1778, 1747, 1715]
Stitching 0 and 4 at ('Cr', 'Ni') from [560, 558, 555, 551, 546, 540, 533, 525, 516, 506, 495, 483, 470, 456, 441, 425, 408, 390, 371, 351, 330, 308, 285, 261, 236, 210, 183, 155, 126, 96, 65, 33, 0] to [2804, 2803, 2801, 2798, 2794, 2789, 2783, 2776, 2768, 2759, 2749, 2738, 2726, 2713, 2699, 2684, 2668, 2651, 2633, 2614, 2594, 2573, 2551, 2528, 2504, 2479, 2453, 2426, 2398, 2369, 2339, 2308, 2276]
Stitching 3 and 4 at ('Cr', 'Ni') from [2243, 2242, 2240, 2237, 2233, 2228, 2222, 2215, 2207, 2198, 2188, 2177, 2165, 2152, 2138, 2123, 2107, 2090, 2072, 2053, 2033, 2012, 1990, 1967, 1943, 1918, 

Stitching 1 and 3 at ('Cr', 'Ti') from [1121, 1119, 1116, 1112, 1107, 1101, 1094, 1086, 1077, 1067, 1056, 1044, 1031, 1017, 1002, 986, 969, 951, 932, 912, 891, 869, 846, 822, 797, 771, 744, 716, 687, 657, 626, 594, 561] to [2243, 2241, 2238, 2234, 2229, 2223, 2216, 2208, 2199, 2189, 2178, 2166, 2153, 2139, 2124, 2108, 2091, 2073, 2054, 2034, 2013, 1991, 1968, 1944, 1919, 1893, 1866, 1838, 1809, 1779, 1748, 1716, 1683]
Stitching 1 and 5 at ('Cr', 'Ti') from [1121, 1119, 1116, 1112, 1107, 1101, 1094, 1086, 1077, 1067, 1056, 1044, 1031, 1017, 1002, 986, 969, 951, 932, 912, 891, 869, 846, 822, 797, 771, 744, 716, 687, 657, 626, 594, 561] to [3365, 3364, 3362, 3359, 3355, 3350, 3344, 3337, 3329, 3320, 3310, 3299, 3287, 3274, 3260, 3245, 3229, 3212, 3194, 3175, 3155, 3134, 3112, 3089, 3065, 3040, 3014, 2987, 2959, 2930, 2900, 2869, 2837]


Stitching 3 and 5 at ('Cr', 'Ti') from [2243, 2241, 2238, 2234, 2229, 2223, 2216, 2208, 2199, 2189, 2178, 2166, 2153, 2139, 2124, 2108, 2091, 2073, 2054, 2034, 2013, 1991, 1968, 1944, 1919, 1893, 1866, 1838, 1809, 1779, 1748, 1716, 1683] to [3365, 3364, 3362, 3359, 3355, 3350, 3344, 3337, 3329, 3320, 3310, 3299, 3287, 3274, 3260, 3245, 3229, 3212, 3194, 3175, 3155, 3134, 3112, 3089, 3065, 3040, 3014, 2987, 2959, 2930, 2900, 2869, 2837]
Stitching 1 and 6 at ('Fe', 'Ti') from [593, 592, 591, 590, 589, 588, 587, 586, 585, 584, 583, 582, 581, 580, 579, 578, 577, 576, 575, 574, 573, 572, 571, 570, 569, 568, 567, 566, 565, 564, 563, 562, 561] to [3926, 3924, 3921, 3917, 3912, 3906, 3899, 3891, 3882, 3872, 3861, 3849, 3836, 3822, 3807, 3791, 3774, 3756, 3737, 3717, 3696, 3674, 3651, 3627, 3602, 3576, 3549, 3521, 3492, 3462, 3431, 3399, 3366]


Stitching 1 and 8 at ('Fe', 'Ti') from [593, 592, 591, 590, 589, 588, 587, 586, 585, 584, 583, 582, 581, 580, 579, 578, 577, 576, 575, 574, 573, 572, 571, 570, 569, 568, 567, 566, 565, 564, 563, 562, 561] to [5048, 5047, 5045, 5042, 5038, 5033, 5027, 5020, 5012, 5003, 4993, 4982, 4970, 4957, 4943, 4928, 4912, 4895, 4877, 4858, 4838, 4817, 4795, 4772, 4748, 4723, 4697, 4670, 4642, 4613, 4583, 4552, 4520]
Stitching 6 and 8 at ('Fe', 'Ti') from [3926, 3924, 3921, 3917, 3912, 3906, 3899, 3891, 3882, 3872, 3861, 3849, 3836, 3822, 3807, 3791, 3774, 3756, 3737, 3717, 3696, 3674, 3651, 3627, 3602, 3576, 3549, 3521, 3492, 3462, 3431, 3399, 3366] to [5048, 5047, 5045, 5042, 5038, 5033, 5027, 5020, 5012, 5003, 4993, 4982, 4970, 4957, 4943, 4928, 4912, 4895, 4877, 4858, 4838, 4817, 4795, 4772, 4748, 4723, 4697, 4670, 4642, 4613, 4583, 4552, 4520]
Stitching 2 and 4 at ('Cr', 'V') from [1682, 1680, 1677, 1673, 1668, 1662, 1655, 1647, 1638, 1628, 1617, 1605, 1592, 1578, 1563, 1547, 1530, 1512, 1493, 

Stitching 2 and 5 at ('Cr', 'V') from [1682, 1680, 1677, 1673, 1668, 1662, 1655, 1647, 1638, 1628, 1617, 1605, 1592, 1578, 1563, 1547, 1530, 1512, 1493, 1473, 1452, 1430, 1407, 1383, 1358, 1332, 1305, 1277, 1248, 1218, 1187, 1155, 1122] to [3365, 3363, 3360, 3356, 3351, 3345, 3338, 3330, 3321, 3311, 3300, 3288, 3275, 3261, 3246, 3230, 3213, 3195, 3176, 3156, 3135, 3113, 3090, 3066, 3041, 3015, 2988, 2960, 2931, 2901, 2870, 2838, 2805]
Stitching 4 and 5 at ('Cr', 'V') from [2804, 2802, 2799, 2795, 2790, 2784, 2777, 2769, 2760, 2750, 2739, 2727, 2714, 2700, 2685, 2669, 2652, 2634, 2615, 2595, 2574, 2552, 2529, 2505, 2480, 2454, 2427, 2399, 2370, 2340, 2309, 2277, 2244] to [3365, 3363, 3360, 3356, 3351, 3345, 3338, 3330, 3321, 3311, 3300, 3288, 3275, 3261, 3246, 3230, 3213, 3195, 3176, 3156, 3135, 3113, 3090, 3066, 3041, 3015, 2988, 2960, 2931, 2901, 2870, 2838, 2805]
Stitching 2 and 7 at ('Fe', 'V') from [1154, 1153, 1152, 1151, 1150, 1149, 1148, 1147, 1146, 1145, 1144, 1143, 1142, 1141,

Stitching 6 and 9 at ('Ni', 'Ti') from [3398, 3397, 3396, 3395, 3394, 3393, 3392, 3391, 3390, 3389, 3388, 3387, 3386, 3385, 3384, 3383, 3382, 3381, 3380, 3379, 3378, 3377, 3376, 3375, 3374, 3373, 3372, 3371, 3370, 3369, 3368, 3367, 3366] to [5609, 5608, 5606, 5603, 5599, 5594, 5588, 5581, 5573, 5564, 5554, 5543, 5531, 5518, 5504, 5489, 5473, 5456, 5438, 5419, 5399, 5378, 5356, 5333, 5309, 5284, 5258, 5231, 5203, 5174, 5144, 5113, 5081]
Stitching 4 and 7 at ('Ni', 'V') from [2276, 2275, 2274, 2273, 2272, 2271, 2270, 2269, 2268, 2267, 2266, 2265, 2264, 2263, 2262, 2261, 2260, 2259, 2258, 2257, 2256, 2255, 2254, 2253, 2252, 2251, 2250, 2249, 2248, 2247, 2246, 2245, 2244] to [3959, 3958, 3957, 3956, 3955, 3954, 3953, 3952, 3951, 3950, 3949, 3948, 3947, 3946, 3945, 3944, 3943, 3942, 3941, 3940, 3939, 3938, 3937, 3936, 3935, 3934, 3933, 3932, 3931, 3930, 3929, 3928, 3927]


Stitching 4 and 9 at ('Ni', 'V') from [2276, 2275, 2274, 2273, 2272, 2271, 2270, 2269, 2268, 2267, 2266, 2265, 2264, 2263, 2262, 2261, 2260, 2259, 2258, 2257, 2256, 2255, 2254, 2253, 2252, 2251, 2250, 2249, 2248, 2247, 2246, 2245, 2244] to [5609, 5607, 5604, 5600, 5595, 5589, 5582, 5574, 5565, 5555, 5544, 5532, 5519, 5505, 5490, 5474, 5457, 5439, 5420, 5400, 5379, 5357, 5334, 5310, 5285, 5259, 5232, 5204, 5175, 5145, 5114, 5082, 5049]


Stitching 7 and 9 at ('Ni', 'V') from [3959, 3958, 3957, 3956, 3955, 3954, 3953, 3952, 3951, 3950, 3949, 3948, 3947, 3946, 3945, 3944, 3943, 3942, 3941, 3940, 3939, 3938, 3937, 3936, 3935, 3934, 3933, 3932, 3931, 3930, 3929, 3928, 3927] to [5609, 5607, 5604, 5600, 5595, 5589, 5582, 5574, 5565, 5555, 5544, 5532, 5519, 5505, 5490, 5474, 5457, 5439, 5420, 5400, 5379, 5357, 5334, 5310, 5285, 5259, 5232, 5204, 5175, 5145, 5114, 5082, 5049]
Stitching 5 and 8 at ('Ti', 'V') from [2837, 2836, 2835, 2834, 2833, 2832, 2831, 2830, 2829, 2828, 2827, 2826, 2825, 2824, 2823, 2822, 2821, 2820, 2819, 2818, 2817, 2816, 2815, 2814, 2813, 2812, 2811, 2810, 2809, 2808, 2807, 2806, 2805] to [4520, 4519, 4518, 4517, 4516, 4515, 4514, 4513, 4512, 4511, 4510, 4509, 4508, 4507, 4506, 4505, 4504, 4503, 4502, 4501, 4500, 4499, 4498, 4497, 4496, 4495, 4494, 4493, 4492, 4491, 4490, 4489, 4488]
Stitching 5 and 9 at ('Ti', 'V') from [2837, 2836, 2835, 2834, 2833, 2832, 2831, 2830, 2829, 2828, 2827, 2826, 2825, 2824,

In [47]:
from pycalphad import Database

# List of database file paths
db_files = [
    "ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB",
    "ammap/databases/Cr-Fe-Ti_wang2017.tdb",
    "ammap/databases/Cr-Fe-Ni_miettinen1999.tdb",
    "ammap/databases/Cr-Ni-Ti_huang2018.tdb",
    "ammap/databases/Cr-Ti-V_ghosh2002.tdb",
    "ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb",
    "ammap/databases/Fe-Ni-V_zhao2014.tdb",
    "ammap/databases/Fe-Ti-V_guo2012.TDB",
    "ammap/databases/Ni-Ti-V_zou2018.tdb"
]

# Dictionary to store unique phases for each database
unique_phases = {}

# Iterate through each database file
for db_file in db_files:
    dbf = Database(db_file)
    phases = list(set(dbf.phases.keys()))
    unique_phases[db_file] = phases

# Print unique phases for each database
for db_file, phases in unique_phases.items():
    print(f"Unique phases for {db_file}: {phases}")

Unique phases for ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB: ['HIGH_SIGMA', 'BCC_A2', 'COV3', 'FCC_A1', 'B2_BCC', 'HCP_A3', 'M3V', 'NI2V', 'L12_FCC', 'NI2V7', 'LIQUID', 'SIGMA']
Unique phases for ammap/databases/Cr-Fe-Ti_wang2017.tdb: ['BCC_B2', 'TI5CR7FE17', 'C15', 'BCC_A2', 'FCC_A1', 'HCP_A3', 'C36', 'LIQUID', 'SIGMA', 'C14']
Unique phases for ammap/databases/Cr-Fe-Ni_miettinen1999.tdb: ['BCC_A2', 'FCC_A1', 'HCP_A3', 'LIQUID', 'SIGMA']
Unique phases for ammap/databases/Cr-Ni-Ti_huang2018.tdb: ['NITI', 'LAVES_C36', 'BCC_A2', 'FCC_A1', 'HCP_A3', 'NITI2', 'LAVES_C14', 'NI3TI', 'LIQUID', 'LAVES_C15']
Unique phases for ammap/databases/Cr-Ti-V_ghosh2002.tdb: ['LAVES_C36', 'BCC_A2', 'HCP_A3', 'LAVES_C14', 'LIQUID', 'LAVES_C15']
Unique phases for ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb: ['A3', 'NITI2', 'NI3TI', 'A1', 'A2', 'LIQUID', 'BCC2', 'C14', 'FCC4']
Unique phases for ammap/databases/Fe-Ni-V_zhao2014.tdb: ['FE4N', 'A2_BCC', 'BCC_A2', 'FCC_A1', 'A15_NI2V7', 'HCP_A3', 'NI2V', 'CBCC_A12

In [48]:

#dbf = Database("CrHfMoNbTaTiVWZr_9element_Feb2023.tdb")
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')
#from myPycalphadCallable import equilibrium_callable

['Cr', 'Fe', 'Ni', 'Ti', 'V']
Loaded TDB file with phases considered: ['SIGMA', 'B2', 'DIAMOND_A4', 'BCC_A2', 'FCC_A1', 'HCP_A3', 'NI3TI', 'NI2V', 'NITI2', 'NIV3', 'LAVES_C14', 'CUB_A13', 'BCT_A5', 'LIQUID', 'CBCC_A12', 'NI3V', 'LAVES_C15']


In [49]:
import os
import importlib

# Directory containing the equilibrium files
directory = "ammap/callables/multi_system_equilibrium_and_scheil"

# Get all files starting with "equilibrium"
equilibrium_files = [f for f in os.listdir(directory) if f.startswith("equilibrium") and f.endswith(".py")]

# Dictionary to store imported callables with unique names
equilibrium_callables = {}

# Import each equilibrium file and store the callable with a unique name
for file in equilibrium_files:
    module_name = file[:-3]  # Remove the .py extension
    module_path = f"ammap.callables.multi_system_equilibrium_and_scheil.{module_name}"
    module = importlib.import_module(module_path)
    callable_name = f"{module_name}"
    equilibrium_callables[callable_name] = getattr(module, "equilibrium_callable")

# Print the imported callables
for name, func in equilibrium_callables.items():
    print(f"Imported {name}: {func}")

/ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/pycalphad/model.py:1278: UserWarning: The order-disorder model for "FCC4" has a contribution from the physical property model `magnetic_energy`. Partitioned physical properties are not correctly substituted into the disordered part of the energy. THE GIBBS ENERGY CALCULATED FOR THIS PHASE MAY BE INCORRECT. Please see the discussion in https://github.com/pycalphad/pycalphad/pull/311 for more details.
  warnings.warn(


Imported equilibrium_callable_CrFeTi_08bbfb9a: <function equilibrium_callable at 0x14fff9f36480>
Imported equilibrium_callable_CrFeV_ab1edb07: <function equilibrium_callable at 0x15000343ab60>
Imported equilibrium_callable_CrNiTi_a9f6f2ff: <function equilibrium_callable at 0x14fff9f77e20>
Imported equilibrium_callable_NiTiV_1d83c99c: <function equilibrium_callable at 0x14fff9fcbc40>
Imported equilibrium_callable_NiCrV_b7aba9ab: <function equilibrium_callable at 0x14fff9fcb9c0>
Imported equilibrium_callable_NiCrFe_f434a6d9: <function equilibrium_callable at 0x14fff91847c0>
Imported equilibrium_callable_FeTiV_fa95b3ee: <function equilibrium_callable at 0x14fff99f89a0>
Imported equilibrium_callable_FeNiV_b9b0384d: <function equilibrium_callable at 0x14fff9113ec0>
Imported equilibrium_callable_CrTiV_ed4c332b: <function equilibrium_callable at 0x14fff8bc7ba0>
Imported equilibrium_callable_FeNiTi_06a49695: <function equilibrium_callable at 0x14fff8521800>


In [50]:
# Get all files starting with "scheil"
scheil_files = [f for f in os.listdir(directory) if f.startswith("scheil") and f.endswith(".py")]
scheil_callables = {}

for file in scheil_files:
    module_name = file[:-3]
    module_path = f"ammap.callables.multi_system_equilibrium_and_scheil.{module_name}"
    module = importlib.import_module(module_path)
    callable_name = f"{module_name}"
    scheil_callables[callable_name] = getattr(module, "scheil_callable")

for name, func in scheil_callables.items():
    print(f"Imported {name}: {func}")


/ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/pycalphad/model.py:1278: UserWarning: The order-disorder model for "FCC4" has a contribution from the physical property model `magnetic_energy`. Partitioned physical properties are not correctly substituted into the disordered part of the energy. THE GIBBS ENERGY CALCULATED FOR THIS PHASE MAY BE INCORRECT. Please see the discussion in https://github.com/pycalphad/pycalphad/pull/311 for more details.
  warnings.warn(


/ocean/projects/dmr190011p/arichte1/micromamba/envs/v2ammap/lib/python3.11/site-packages/pycalphad/io/tdb.py:293: UserWarning: The type definition character `&` in `TYPE_DEFINITION & GES A_P_D A2_BCC MAGNETIC -1.0 4.00000E-01 ` is not used by any phase.
  warnings.warn(f"The type definition character `{typechar}` in `TYPE_DEFINITION {typechar} {line}` is not used by any phase.")


Imported scheil_callable_FeNiTi_06a49695: <function scheil_callable at 0x14fff8daec00>
Imported scheil_callable_NiCrFe_f434a6d9: <function scheil_callable at 0x14fff85ecae0>
Imported scheil_callable_CrNiTi_a9f6f2ff: <function scheil_callable at 0x14fff8520540>
Imported scheil_callable_FeNiV_b9b0384d: <function scheil_callable at 0x14fff5a61440>
Imported scheil_callable_CrFeV_ab1edb07: <function scheil_callable at 0x14fff5a632e0>
Imported scheil_callable_NiCrV_b7aba9ab: <function scheil_callable at 0x14fff55556c0>
Imported scheil_callable_FeTiV_fa95b3ee: <function scheil_callable at 0x14fff4f65e40>
Imported scheil_callable_CrTiV_ed4c332b: <function scheil_callable at 0x14fff4ec7a60>
Imported scheil_callable_NiTiV_1d83c99c: <function scheil_callable at 0x14fff4ec7b00>
Imported scheil_callable_CrFeTi_08bbfb9a: <function scheil_callable at 0x14fff4add300>


In [51]:
def process_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        middle_part = parts[2]
        if middle_part in element_mapping:
            return middle_part, element_mapping[middle_part]
    return None, None

In [52]:
element_mapping=mapping

In [53]:
def process_key(key):
    parts = key.split('_')
    if len(parts) >= 3:
        middle_part = parts[2]
        if middle_part in mapping:
            return middle_part, mapping[middle_part]['id'], mapping[middle_part]['elements']
        
        # If direct matching fails, try matching by elements
        middle_elements = set(middle_part[i:i+2] for i in range(0, len(middle_part), 2))
        for map_key, value in mapping.items():
            if set(value['elements']) == middle_elements:
                return middle_part, value['id'], value['elements']
    
    return None, None, None

# Process each key in the equilibrium_callables dictionary
id_to_callable = {}
for key in equilibrium_callables:
    middle_part, mapping_id, elements = process_key(key)
    if middle_part and mapping_id is not None:
        print(f"Key: {key}")
        print(f"The mapping number for {middle_part} is {mapping_id}")
        print(f"Elements: {elements}")
        print("---")
        id_to_callable[mapping_id] = key
    else:
        print(f"No mapping found for key: {key}")
        print("---")

sc_id_to_callable = {}
for key in scheil_callables:
    middle_part, mapping_id, elements = process_key(key)
    if middle_part and mapping_id is not None:
        print(f"Key: {key}")
        print(f"The mapping number for {middle_part} is {mapping_id}")
        print(f"Elements: {elements}")
        print("---")
        sc_id_to_callable[mapping_id] = key
    else:
        print(f"No mapping found for key: {key}")
        print("---")

Key: equilibrium_callable_CrFeTi_08bbfb9a
The mapping number for CrFeTi is 1
Elements: ['Cr', 'Fe', 'Ti']
---
Key: equilibrium_callable_CrFeV_ab1edb07
The mapping number for CrFeV is 2
Elements: ['Cr', 'Fe', 'V']
---
Key: equilibrium_callable_CrNiTi_a9f6f2ff
The mapping number for CrNiTi is 3
Elements: ['Cr', 'Ni', 'Ti']
---
Key: equilibrium_callable_NiTiV_1d83c99c
The mapping number for NiTiV is 9
Elements: ['Ni', 'Ti', 'V']
---
Key: equilibrium_callable_NiCrV_b7aba9ab
The mapping number for NiCrV is 4
Elements: ['Cr', 'Ni', 'V']
---
Key: equilibrium_callable_NiCrFe_f434a6d9
The mapping number for NiCrFe is 0
Elements: ['Cr', 'Fe', 'Ni']
---
Key: equilibrium_callable_FeTiV_fa95b3ee
The mapping number for FeTiV is 8
Elements: ['Fe', 'Ti', 'V']
---
Key: equilibrium_callable_FeNiV_b9b0384d
The mapping number for FeNiV is 7
Elements: ['Fe', 'Ni', 'V']
---
Key: equilibrium_callable_CrTiV_ed4c332b
The mapping number for CrTiV is 5
Elements: ['Cr', 'Ti', 'V']
---
Key: equilibrium_callable_Fe

In [54]:
print(id_to_callable)
print(sc_id_to_callable)

{1: 'equilibrium_callable_CrFeTi_08bbfb9a', 2: 'equilibrium_callable_CrFeV_ab1edb07', 3: 'equilibrium_callable_CrNiTi_a9f6f2ff', 9: 'equilibrium_callable_NiTiV_1d83c99c', 4: 'equilibrium_callable_NiCrV_b7aba9ab', 0: 'equilibrium_callable_NiCrFe_f434a6d9', 8: 'equilibrium_callable_FeTiV_fa95b3ee', 7: 'equilibrium_callable_FeNiV_b9b0384d', 5: 'equilibrium_callable_CrTiV_ed4c332b', 6: 'equilibrium_callable_FeNiTi_06a49695'}
{6: 'scheil_callable_FeNiTi_06a49695', 0: 'scheil_callable_NiCrFe_f434a6d9', 3: 'scheil_callable_CrNiTi_a9f6f2ff', 7: 'scheil_callable_FeNiV_b9b0384d', 2: 'scheil_callable_CrFeV_ab1edb07', 4: 'scheil_callable_NiCrV_b7aba9ab', 8: 'scheil_callable_FeTiV_fa95b3ee', 5: 'scheil_callable_CrTiV_ed4c332b', 9: 'scheil_callable_NiTiV_1d83c99c', 1: 'scheil_callable_CrFeTi_08bbfb9a'}


In [55]:
compositions_with_id[:5]

[([0.0, 0.0, 1.0, 0.0, 0.0], 0),
 ([0.0, 0.03125, 0.96875, 0.0, 0.0], 0),
 ([0.0, 0.0625, 0.9375, 0.0, 0.0], 0),
 ([0.0, 0.09375, 0.90625, 0.0, 0.0], 0),
 ([0.0, 0.125, 0.875, 0.0, 0.0], 0)]

In [56]:
mapping

{'CrFeNi': {'id': 0, 'elements': ['Cr', 'Fe', 'Ni']},
 'CrFeTi': {'id': 1, 'elements': ['Cr', 'Fe', 'Ti']},
 'CrFeV': {'id': 2, 'elements': ['Cr', 'Fe', 'V']},
 'CrNiTi': {'id': 3, 'elements': ['Cr', 'Ni', 'Ti']},
 'CrNiV': {'id': 4, 'elements': ['Cr', 'Ni', 'V']},
 'CrTiV': {'id': 5, 'elements': ['Cr', 'Ti', 'V']},
 'FeNiTi': {'id': 6, 'elements': ['Fe', 'Ni', 'Ti']},
 'FeNiV': {'id': 7, 'elements': ['Fe', 'Ni', 'V']},
 'FeTiV': {'id': 8, 'elements': ['Fe', 'Ti', 'V']},
 'NiTiV': {'id': 9, 'elements': ['Ni', 'Ti', 'V']}}

In [57]:
def reduce_compositions(compositions_with_id, mapping):
    element_order = elementalSpaceComponents
    system_comps_with_id = []
    reduced_compositions = []
    
    for composition, comp_id in compositions_with_id:
        relevant_entry = next((entry for entry in mapping.values() if entry['id'] == comp_id), None)
        if not relevant_entry:
            continue
        
        relevant_indices = [element_order.index(elem) for elem in relevant_entry['elements']]
        reduced_point = [composition[index] for index in relevant_indices]
        system_comps_with_id.append((reduced_point, comp_id))
        reduced_compositions.append(reduced_point)
    
    return system_comps_with_id, reduced_compositions

In [58]:
system_comps_with_id, reduced_compositions = reduce_compositions(compositions_with_id, mapping)

In [59]:
from tqdm.contrib.concurrent import process_map

In [60]:
#startingNodes = [0, 90, 200, 310] + random.sample(range(len(reduced_compositions)), 11)
startingNodes = [0, 90, 20, 31] + random.sample(range(len(reduced_compositions)), 11)
print(f"Starting nodes: {startingNodes}")

for startingNode in startingNodes:
    print(f"Starting node: {reduced_compositions[startingNode]}")

Starting nodes: [0, 90, 20, 31, 428, 2192, 714, 3336, 2183, 882, 312, 3105, 4392, 4606, 2721]
Starting node: [0.0, 0.0, 1.0]
Starting node: [0.0625, 0.78125, 0.15625]
Starting node: [0.0, 0.625, 0.375]
Starting node: [0.0, 0.96875, 0.03125]
Starting node: [0.53125, 0.09375, 0.375]
Starting node: [0.71875, 0.09375, 0.1875]
Starting node: [0.125, 0.84375, 0.03125]
Starting node: [0.78125, 0.1875, 0.03125]
Starting node: [0.6875, 0.15625, 0.15625]
Starting node: [0.34375, 0.40625, 0.25]
Starting node: [0.34375, 0.125, 0.53125]
Starting node: [0.3125, 0.46875, 0.21875]
Starting node: [0.59375, 0.28125, 0.125]
Starting node: [0.09375, 0.6875, 0.21875]
Starting node: [0.625, 0.21875, 0.15625]


In [61]:
gridFeasible = [None]*len(reduced_compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

In [62]:
# Function to get the correct callable for a given composition
def get_callable(composition):
    for comp, id in compositions_with_id:
        if comp == composition:
            callable_name = id_to_callable.get(id)
            if callable_name is None:
                raise ValueError(f"No callable name found for ID {id}")
            if callable_name in globals():
                return globals()[callable_name]
            else:
                raise NameError(f"Function '{callable_name}' not found in global scope")
    raise ValueError(f"No callable found for composition {composition}")

def process_composition(elP):
    try:
        callable_func = get_callable(elP)
        return callable_func(elP)
    except Exception as e:
        print(f"Error processing composition {elP}: {str(e)}")
        return None


In [63]:
# Function to get the correct callable for a given composition
def get_sc_callable(composition):
    for comp, id in compositions_with_id:
        if comp == composition:
            callable_name = sc_id_to_callable.get(id)
            if callable_name is None:
                raise ValueError(f"No callable name found for ID {id}")
            if callable_name in globals():
                return globals()[callable_name]
            else:
                raise NameError(f"Function '{callable_name}' not found in global scope")
    raise ValueError(f"No callable found for composition {composition}")

def process_sc_composition(elP):
    try:
        callable_func = get_sc_callable(elP)
        return callable_func(elP)
    except Exception as e:
        print(f"Error processing composition {elP}: {str(e)}")
        return None

In [64]:
print(equilibrium_callables)
print(scheil_callables)

{'equilibrium_callable_CrFeTi_08bbfb9a': <function equilibrium_callable at 0x14fff9f36480>, 'equilibrium_callable_CrFeV_ab1edb07': <function equilibrium_callable at 0x15000343ab60>, 'equilibrium_callable_CrNiTi_a9f6f2ff': <function equilibrium_callable at 0x14fff9f77e20>, 'equilibrium_callable_NiTiV_1d83c99c': <function equilibrium_callable at 0x14fff9fcbc40>, 'equilibrium_callable_NiCrV_b7aba9ab': <function equilibrium_callable at 0x14fff9fcb9c0>, 'equilibrium_callable_NiCrFe_f434a6d9': <function equilibrium_callable at 0x14fff91847c0>, 'equilibrium_callable_FeTiV_fa95b3ee': <function equilibrium_callable at 0x14fff99f89a0>, 'equilibrium_callable_FeNiV_b9b0384d': <function equilibrium_callable at 0x14fff9113ec0>, 'equilibrium_callable_CrTiV_ed4c332b': <function equilibrium_callable at 0x14fff8bc7ba0>, 'equilibrium_callable_FeNiTi_06a49695': <function equilibrium_callable at 0x14fff8521800>}
{'scheil_callable_FeNiTi_06a49695': <function scheil_callable at 0x14fff8daec00>, 'scheil_calla

In [65]:
sampled_compositions = compositions_with_id[::50]

# Print the sampled compositions to verify
for composition in sampled_compositions:
    print(composition)

([0.0, 0.0, 1.0, 0.0, 0.0], 0)
([0.03125, 0.53125, 0.4375, 0.0, 0.0], 0)
([0.09375, 0.125, 0.78125, 0.0, 0.0], 0)
([0.125, 0.75, 0.125, 0.0, 0.0], 0)
([0.1875, 0.53125, 0.28125, 0.0, 0.0], 0)
([0.25, 0.4375, 0.3125, 0.0, 0.0], 0)
([0.3125, 0.46875, 0.21875, 0.0, 0.0], 0)
([0.375, 0.625, 0.0, 0.0, 0.0], 0)
([0.46875, 0.3125, 0.21875, 0.0, 0.0], 0)
([0.5625, 0.28125, 0.15625, 0.0, 0.0], 0)
([0.6875, 0.15625, 0.15625, 0.0, 0.0], 0)
([0.875, 0.125, 0.0, 0.0, 0.0], 0)
([0.03125, 0.1875, 0.0, 0.78125, 0.0], 1)
([0.0625, 0.75, 0.0, 0.1875, 0.0], 1)
([0.125, 0.40625, 0.0, 0.46875, 0.0], 1)
([0.1875, 0.1875, 0.0, 0.625, 0.0], 1)
([0.25, 0.09375, 0.0, 0.65625, 0.0], 1)
([0.3125, 0.125, 0.0, 0.5625, 0.0], 1)
([0.375, 0.28125, 0.0, 0.34375, 0.0], 1)
([0.4375, 0.5625, 0.0, 0.0, 0.0], 1)
([0.53125, 0.4375, 0.0, 0.03125, 0.0], 1)
([0.65625, 0.1875, 0.0, 0.15625, 0.0], 1)
([0.8125, 0.1875, 0.0, 0.0, 0.0], 1)
([0.0, 0.875, 0.0, 0.0, 0.125], 2)
([0.0625, 0.40625, 0.0, 0.0, 0.53125], 2)
([0.125, 0.0625, 

In [66]:
print(equilibrium_callables['equilibrium_callable_CrNiTi_a9f6f2ff'](compositions[200]))
compositions[200]

{'Phases': ['NITI', 'NI3TI', 'NITI'], 'PhaseFraction': [0.18227786643860125, 0.4884579802388515, 0.329264153353066]}


[0.1875, 0.53125, 0.28125, 0.0, 0.0]

In [67]:
from functools import partial

def get_equilibrium_callable(composition, id_to_callable, equilibrium_callables):
    composition_id = composition[1]  # Get the ID from the composition tuple
    #print(f"Composition ID: {composition_id}")
    callable_name = id_to_callable.get(composition_id)
    if callable_name is None:
        raise ValueError(f"No callable found for composition ID {composition_id}")
    callable_func = equilibrium_callables.get(callable_name)
    if callable_func is None:
        raise ValueError(f"No callable function found for name {callable_name}")
    return callable_func

def apply_equilibrium_callable(callable_and_position):
    callable_func, position = callable_and_position
    return callable_func(position)

In [68]:
from functools import partial

def get_scheil_callable(composition, sc_id_to_callable, scheil_callables):
    composition_id = composition[1]  # Get the ID from the composition tuple
    #print(f"Composition ID: {composition_id}")
    callable_name = sc_id_to_callable.get(composition_id)
    if callable_name is None:
        raise ValueError(f"No callable found for composition ID {composition_id}")
    callable_func = scheil_callables.get(callable_name)
    if callable_func is None:
        raise ValueError(f"No callable function found for name {callable_name}")
    return callable_func

def apply_scheil_callable(callable_and_position):
    callable_func, position = callable_and_position
    return callable_func(position)

#WORKING
while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of equilibrium callables for each composition
    equilibrium_callables_list = [get_equilibrium_callable(system_comps_with_id[i], id_to_callable, equilibrium_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_equilibrium_callable, zip(equilibrium_callables_list, elPositions), max_workers=4)
    else:
        results = [ec(elP) for ec, elP in zip(equilibrium_callables_list, elPositions)]
    
    # Extract only the 'Phases' component from the results
    phases = [result['Phases'] for result in results]
    
    feasibilities = [len(set(p) & set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC','A2_FCC'])) == 0 and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

In [69]:
import json

# Initialize a list to store the results
results_list = []

while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of equilibrium callables for each composition
    equilibrium_callables_list = [get_equilibrium_callable(system_comps_with_id[i], id_to_callable, equilibrium_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_equilibrium_callable, zip(equilibrium_callables_list, elPositions), max_workers=4)
    else:
        results = [ec(elP) for ec, elP in zip(equilibrium_callables_list, elPositions)]
    
    # Extract only the 'Phases' component from the results
    phases = [result['Phases'] for result in results]
    
    #feasibilities = [len(set(p) & set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC','A2_FCC','L12_FCC','BCC2', 'A1', 'A2', 'A3', 'FCC4'])) == 0 and p != [] for p in phases]
    feasibilities = [set(p).issubset(set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC', 'A2_FCC', 'L12_FCC', 'BCC2', 'A1', 'A2', 'A3', 'FCC4'])) and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Save the current step result and elPositions
    results_list.append({
        'queue': queue,
        'elPositions': elPositions,
        'results': results
    })

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

# Write the results to a JSON file
with open('results.json', 'w') as f:
    json.dump(results_list, f, indent=4)

Queue: [0, 90, 20, 31, 428, 2192, 714, 3336, 2183, 882, 312, 3105, 4392, 4606, 2721]
[[0.0, 0.0, 1.0], [0.0625, 0.78125, 0.15625], [0.0, 0.625, 0.375], [0.0, 0.96875, 0.03125], [0.53125, 0.09375, 0.375], [0.71875, 0.09375, 0.1875], [0.125, 0.84375, 0.03125], [0.78125, 0.1875, 0.03125], [0.6875, 0.15625, 0.15625], [0.34375, 0.40625, 0.25], [0.34375, 0.125, 0.53125], [0.3125, 0.46875, 0.21875], [0.59375, 0.28125, 0.125], [0.09375, 0.6875, 0.21875], [0.625, 0.21875, 0.15625]]


0it [00:00, ?it/s]

Calculations done: 15    | Explored points: 23   
Queue: [1, 4483, 4485, 3847, 19, 21, 4377, 4378, 411, 412, 30, 32, 33, 290, 289, 4391, 4393, 4395, 427, 429, 1714, 52, 53, 4405, 311, 4408, 313, 58, 59, 443, 444, 4406, 63, 64, 333, 334, 3922, 3924, 89, 91, 2275, 120, 121, 3834]
[[0.0, 0.03125, 0.96875], [0.9375, 0.03125, 0.03125], [0.96875, 0.0, 0.03125], [0.625, 0.34375, 0.03125], [0.0, 0.59375, 0.40625], [0.0, 0.65625, 0.34375], [0.5625, 0.28125, 0.15625], [0.5625, 0.3125, 0.125], [0.5, 0.09375, 0.40625], [0.5, 0.125, 0.375], [0.0, 0.9375, 0.0625], [0.0, 1.0, 0.0], [0.03125, 0.0, 0.96875], [0.3125, 0.15625, 0.53125], [0.3125, 0.125, 0.5625], [0.59375, 0.25, 0.15625], [0.59375, 0.3125, 0.09375], [0.59375, 0.375, 0.03125], [0.53125, 0.0625, 0.40625], [0.53125, 0.125, 0.34375], [0.0, 0.96875, 0.03125], [0.03125, 0.59375, 0.375], [0.03125, 0.625, 0.34375], [0.625, 0.25, 0.125], [0.34375, 0.09375, 0.5625], [0.625, 0.34375, 0.03125], [0.34375, 0.15625, 0.5], [0.03125, 0.78125, 0.1875], [0.

0it [00:00, ?it/s]

Calculations done: 59    | Explored points: 89   
Queue: [2, 18, 22, 25, 26, 27, 29, 34, 51, 54, 57, 60, 62, 65, 592, 83, 84, 85, 88, 92, 94, 95, 624, 625, 119, 122, 1153, 149, 150, 151, 1185, 1713, 1746, 2274, 3820, 2307, 4361, 4362, 4363, 267, 266, 265, 3859, 4376, 4379, 4380, 4381, 288, 291, 4390, 4394, 4404, 310, 4407, 314, 4417, 4418, 4419, 4420, 3397, 332, 3919, 335, 353, 354, 355, 3429, 3958, 4479, 4480, 4482, 4484, 4487, 393, 394, 395, 3990, 410, 413, 426, 430, 5044, 5047, 442, 445, 457, 458, 459, 5605, 5609]
[[0.0, 0.0625, 0.9375], [0.0, 0.5625, 0.4375], [0.0, 0.6875, 0.3125], [0.0, 0.78125, 0.21875], [0.0, 0.8125, 0.1875], [0.0, 0.84375, 0.15625], [0.0, 0.90625, 0.09375], [0.03125, 0.03125, 0.9375], [0.03125, 0.5625, 0.40625], [0.03125, 0.65625, 0.3125], [0.03125, 0.75, 0.21875], [0.03125, 0.84375, 0.125], [0.03125, 0.90625, 0.0625], [0.0625, 0.0, 0.9375], [0.0, 0.96875, 0.03125], [0.0625, 0.5625, 0.375], [0.0625, 0.59375, 0.34375], [0.0625, 0.625, 0.3125], [0.0625, 0.71875, 

...

0it [00:00, ?it/s]

Calculations done: 1842  | Explored points: 2108 
Queue: [5112, 2836, 2869, 4551]
[[0.03125, 0.9375, 0.03125], [0.0, 0.96875, 0.03125], [0.03125, 0.96875, 0.0], [0.03125, 0.9375, 0.03125]]


0it [00:00, ?it/s]

Calculations done: 1846  | Explored points: 2114 
Queue: [4552, 5113, 2837]
[[0.03125, 0.96875, 0.0], [0.03125, 0.96875, 0.0], [0.0, 1.0, 0.0]]


Calculations done: 1849  | Explored points: 2119 


In [70]:

# Load the results from the JSON file
with open('/ocean/projects/dmr190011p/arichte1/github_repo/AMMap/results.json', 'r') as f:
    data = json.load(f)

# Initialize a dictionary to store the merged results
merged_results = {}

# Iterate through each step in the data
for step in data:
    queue = step['queue']
    results = step['results']
    
    # Merge the queue numbers with their associated result values
    for q, result in zip(queue, results):
        merged_results[q] = result

# Convert the merged results dictionary to a list of dictionaries
merged_results_list = [{'queue': q, 'result': result} for q, result in merged_results.items()]

# Save the merged results to a new JSON file
with open('merged_results.json', 'w') as f:
    json.dump(merged_results_list, f, indent=4)
with open('grid_feasible.json', 'w') as f:
    json.dump(gridFeasible, f, indent=4)

In [71]:
gridFeasible1 = [None]*len(reduced_compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

# Initialize a list to store the results
results_list = []

while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of scheil callables for each composition
    scheil_callables_list = [get_scheil_callable(system_comps_with_id[i], sc_id_to_callable, scheil_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_scheil_callable, zip(scheil_callables_list, elPositions), max_workers=4)
    else:
        results = [sc(elP) for sc, elP in zip(scheil_callables_list, elPositions)]
    
    # Extract only the 'Phases' component from the results
    phases = [result['finalPhase'] for result in results]
    
    feasibilities = [set(p).issubset(set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC', 'A2_FCC', 'L12_FCC', 'BCC2', 'A1', 'A2', 'A3', 'FCC4'])) and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Save the current step result and elPositions
    results_list.append({
        'queue': queue,
        'elPositions': elPositions,
        'results': results
    })

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible1[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible1[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

# Write the results to a JSON file
with open('scheil_results.json', 'w') as f:
    json.dump(results_list, f, indent=4)


In [72]:

import logging


# Set up logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

def apply_scheil_callable(args):
    sc, elP = args
    try:
        return sc(elP)
    except Exception as e:
        logging.error(f"Error processing item: {str(e)}")
        return None

# Initialize a list to store the results
results_list = []

while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of scheil callables for each composition
    scheil_callables_list = [get_scheil_callable(system_comps_with_id[i], sc_id_to_callable, scheil_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_scheil_callable, zip(scheil_callables_list, elPositions), max_workers=4)
    else:
        results = [apply_scheil_callable((sc, elP)) for sc, elP in zip(scheil_callables_list, elPositions)]
    
    # Filter out None values (errors) and process valid results
    valid_results = [r for r in results if r is not None]
    
    # Extract only the 'Phases' component from the valid results
    phases = [result['finalPhase'] for result in valid_results]
    
    feasibilities = [set(p).issubset(set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC', 'A2_FCC', 'L12_FCC', 'BCC2', 'A1', 'A2', 'A3', 'FCC4'])) and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Save the current step result and elPositions
    results_list.append({
        'queue': queue,
        'elPositions': elPositions,
        'results': valid_results
    })

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible1[i] = f

        # And equivalent explored points based on system stitching
        explored = explored.union(graphNS[i])
        for eq in graphNS[i]:
            gridFeasible1[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                for n in graphN[eq]:
                    if n not in explored and n not in nextQueuePlusEquivalent:
                        nextQueue.add(n)
                        nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

# Write the results to a JSON file
with open('scheil_results.json', 'w') as f:
    json.dump(results_list, f, indent=4)


Queue: [0, 90, 20, 31, 428, 2192, 714, 3336, 2183, 882, 312, 3105, 4392, 4606, 2721]
[[0.0, 0.0, 1.0], [0.0625, 0.78125, 0.15625], [0.0, 0.625, 0.375], [0.0, 0.96875, 0.03125], [0.53125, 0.09375, 0.375], [0.71875, 0.09375, 0.1875], [0.125, 0.84375, 0.03125], [0.78125, 0.1875, 0.03125], [0.6875, 0.15625, 0.15625], [0.34375, 0.40625, 0.25], [0.34375, 0.125, 0.53125], [0.3125, 0.46875, 0.21875], [0.59375, 0.28125, 0.125], [0.09375, 0.6875, 0.21875], [0.625, 0.21875, 0.15625]]


0it [00:00, ?it/s]

2025-02-07 04:09:32,292 - ERROR - Error processing item: only length-1 arrays can be converted to Python scalars


2025-02-07 04:09:44,587 - ERROR - Error processing item: only length-1 arrays can be converted to Python scalars


Calculations done: 13    | Explored points: 23   
Queue: [1, 4483, 4485, 2182, 3847, 2184, 903, 904, 2193, 2194, 19, 21, 4377, 4378, 411, 412, 30, 32, 33, 4391, 4393, 4395, 427, 429, 685, 684, 1714, 52, 53, 4405, 4406, 4408, 58, 59, 443, 444, 63, 64, 713, 715, 3922, 3924, 89, 91, 860, 859, 2275, 742, 743, 881, 883, 120, 121, 3834, 2171, 2172]
[[0.0, 0.03125, 0.96875], [0.9375, 0.03125, 0.03125], [0.96875, 0.0, 0.03125], [0.6875, 0.125, 0.1875], [0.625, 0.34375, 0.03125], [0.6875, 0.1875, 0.125], [0.375, 0.375, 0.25], [0.375, 0.40625, 0.21875], [0.71875, 0.125, 0.15625], [0.71875, 0.15625, 0.125], [0.0, 0.59375, 0.40625], [0.0, 0.65625, 0.34375], [0.5625, 0.28125, 0.15625], [0.5625, 0.3125, 0.125], [0.5, 0.09375, 0.40625], [0.5, 0.125, 0.375], [0.0, 0.9375, 0.0625], [0.0, 1.0, 0.0], [0.03125, 0.0, 0.96875], [0.59375, 0.25, 0.15625], [0.59375, 0.3125, 0.09375], [0.59375, 0.375, 0.03125], [0.53125, 0.0625, 0.40625], [0.53125, 0.125, 0.34375], [0.09375, 0.875, 0.03125], [0.09375, 0.84375, 

...

Calculations done: 2497  | Explored points: 2807 
Queue: [4518, 4551, 2836, 2869, 5112]
[[0.0, 0.9375, 0.0625], [0.03125, 0.9375, 0.03125], [0.0, 0.96875, 0.03125], [0.03125, 0.96875, 0.0], [0.03125, 0.9375, 0.03125]]


0it [00:00, ?it/s]

Calculations done: 2502  | Explored points: 2813 
Queue: [4552, 5113, 2837, 4519]
[[0.03125, 0.96875, 0.0], [0.03125, 0.96875, 0.0], [0.0, 1.0, 0.0], [0.0, 0.96875, 0.03125]]


0it [00:00, ?it/s]

Calculations done: 2506  | Explored points: 2818 


In [73]:

# Load the results from the JSON file
with open('/ocean/projects/dmr190011p/arichte1/github_repo/AMMap/scheil_results.json', 'r') as f:
    data = json.load(f)

# Initialize a dictionary to store the merged results
merged_results = {}

# Iterate through each step in the data
for step in data:
    queue = step['queue']
    results = step['results']
    
    # Merge the queue numbers with their associated result values
    for q, result in zip(queue, results):
        merged_results[q] = result

# Convert the merged results dictionary to a list of dictionaries
merged_results_list = [{'queue': q, 'result': result} for q, result in merged_results.items()]

# Save the merged results to a new JSON file
with open('merged_scheil_results.json', 'w') as f:
    json.dump(merged_results_list, f, indent=4)
with open('grid_feasible1.json', 'w') as f:
    json.dump(gridFeasible1, f, indent=4)

Out of the explored points, around 1040 should be feasible (i.e., satisfy the equilibrium constraint).

In [74]:
from pathfinding.core.graph import Graph
from pathfinding.finder.dijkstra import DijkstraFinder

In [75]:
pathFindEdges = []
for edge in edges:
    if gridFeasible[edge[0]] and gridFeasible[edge[1]]:
        pathFindEdges.append([edge[0], edge[1], 1])

In [76]:
pathfindingGraph = Graph(edges=pathFindEdges, bi_directional=False)

In [77]:
finder = DijkstraFinder()

In [78]:
len(list(pathfindingGraph.nodes.keys()))

1653

In [79]:
print("Nodes in the graph:", list(pathfindingGraph.nodes.keys()))

Nodes in the graph: [0, 1, 33, 2, 34, 3, 35, 4, 36, 5, 37, 6, 38, 7, 39, 8, 40, 9, 41, 10, 42, 11, 43, 12, 44, 13, 45, 14, 46, 15, 47, 16, 48, 17, 49, 18, 50, 19, 51, 20, 52, 21, 53, 22, 54, 23, 55, 24, 56, 25, 57, 26, 58, 27, 59, 28, 60, 29, 61, 30, 62, 31, 63, 32, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217,

In [80]:
# start_node_id = 194
# end_node_id = 830
start_node_id = 0#12
end_node_id = 408

# Check if the nodes exist in the graph
if start_node_id in pathfindingGraph.nodes and end_node_id in pathfindingGraph.nodes:
    path, runs = finder.find_path(
        pathfindingGraph.node(start_node_id), 
        pathfindingGraph.node(end_node_id), 
        pathfindingGraph)
    print("Path found:", path)
else:
    print(f"One or both of the nodes {start_node_id} and {end_node_id} do not exist in the graph.")

Path found: [<GraphNode(0 0x14ffa8abd410)>, <GraphNode(33 0x150092df1f50)>, <GraphNode(65 0x150092df3150)>, <GraphNode(96 0x150092df3910)>, <GraphNode(126 0x150092e080d0)>, <GraphNode(155 0x150092e08810)>, <GraphNode(183 0x150092e09090)>, <GraphNode(210 0x150092e09750)>, <GraphNode(236 0x150092e09bd0)>, <GraphNode(261 0x150092e09fd0)>, <GraphNode(285 0x150092e0a350)>, <GraphNode(308 0x150092e0a690)>, <GraphNode(330 0x150092e0a9d0)>, <GraphNode(351 0x150092e0acd0)>, <GraphNode(371 0x150092e0afd0)>, <GraphNode(390 0x150092e0b290)>, <GraphNode(408 0x150092e0b550)>]


In [81]:
shortestPath = [p.node_id for p in path]

In [82]:
# Hover approximate formula for each point
formulas = []
for i, comp in enumerate(compositions):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))

# # Generate the projected grid
# gridAtt_projected_df = pd.DataFrame(plotting.simplex2cartesian_py(gridAtt), columns=['x','y','z'])

# # Attach pure component (alloy) labels to corners
# pureComponentIndices = nimplex.pure_component_indexes_py(4, 12)
# labels = ['']*len(gridAtt_projected_df)
# for comp, idx in zip(attainableSpaceComponents, pureComponentIndices):
#     labels[idx] = "<b>"+comp+"</b>"

In [83]:
for step, i in enumerate(shortestPath):
    print(f"{step+1:>2}: {formulas[i]}")

 1: (  0) Ni100.0 
 2: ( 33) Cr3.1 Ni96.9 
 3: ( 65) Cr6.2 Ni93.8 
 4: ( 96) Cr9.4 Ni90.6 
 5: (126) Cr12.5 Ni87.5 
 6: (155) Cr15.6 Ni84.4 
 7: (183) Cr18.8 Ni81.2 
 8: (210) Cr21.9 Ni78.1 
 9: (236) Cr25.0 Ni75.0 
10: (261) Cr28.1 Ni71.9 
11: (285) Cr31.2 Ni68.8 
12: (308) Cr34.4 Ni65.6 
13: (330) Cr37.5 Ni62.5 
14: (351) Cr40.6 Ni59.4 
15: (371) Cr43.8 Ni56.2 
16: (390) Cr46.9 Ni53.1 
17: (408) Cr50.0 Ni50.0 
